In [1]:
# High-quality PyTorch training pipeline tuned for ~4000 images
# Keep dataset paths unchanged.
# Requirements: torch, torchvision, timm, scikit-learn, tqdm, matplotlib, seaborn

import os
import random
import time
import zipfile
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, SWALR
from torchvision import transforms, datasets
import timm
from tqdm import tqdm

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data.sampler import WeightedRandomSampler

# ----------------------------- USER CONFIG -----------------------------
DRIVE_MOUNT = True
ZIP_PATH = "/content/drive/MyDrive/edited.zip"   # if using zipped dataset on Drive
EXTRACT_PATH = "/content/dataset"
TRAIN_DIR = "/content/dataset/edited/train"
VAL_DIR   = "/content/dataset/edited/val"
GRAPH_DIR = "/content/drive/MyDrive/results_4k"
CHECKPOINT_PATH = "/content/drive/MyDrive/best_model_4k.pt"
FINAL_MODEL_PATH = "/content/drive/MyDrive/final_model_4k.pt"

# recommended: use "vit_base_patch16_224" for 4k images if GPU >= 12GB,
# otherwise use EfficientNet-B3/B4 for smaller GPUs
PREFERRED_MODEL = "vit_base_patch16_224"
FALLBACK_MODEL = "efficientnet_b3"
IMG_SIZE = 224

# tuned hyperparameters for ~4k images
BATCH_SIZE = 24
NUM_WORKERS = 4
SEED = 42
PHASE1_EPOCHS = 8    # head-only
PHASE2_EPOCHS = 30   # fine-tune
N_UNFREEZE = 8       # unfreeze last N blocks (ViT) or fine-tune more for CNNs

HEAD_LR = 3e-4
FINETUNE_LR = 3e-5
WEIGHT_DECAY = 1e-2
LABEL_SMOOTHING = 0.05
MIXUP_ALPHA = 0.2
CUTMIX_ALPHA = 1.0
CUTMIX_PROB = 0.4
MAX_GRAD_NORM = 1.0
ACCUMULATION_STEPS = 1
SWA_FINISH_EPOCHS = 6   # SWA duration at end of fine-tuning
EARLY_STOPPING_PATIENCE = 10

# TTA transforms for robust validation
TTA_TRANSFORMS = [
    lambda x: x,
    lambda x: torch.flip(x, dims=[-1]),
    lambda x: torch.rot90(x, k=1, dims=[-2, -1])
]
# -----------------------------------------------------------------------

# reproducibility
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True

# mount drive if available
if DRIVE_MOUNT:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
    except Exception:
        pass

# extract zip if provided
if os.path.exists(ZIP_PATH) and not os.path.exists(EXTRACT_PATH):
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(EXTRACT_PATH)
        print("Extracted dataset to", EXTRACT_PATH)
else:
    print("Dataset zip not found or already extracted (or EXTRACT_PATH exists).")

os.makedirs(GRAPH_DIR, exist_ok=True)
os.makedirs(os.path.dirname(CHECKPOINT_PATH), exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

if not os.path.isdir(TRAIN_DIR) or not os.path.isdir(VAL_DIR):
    raise FileNotFoundError(f"Train/Val directories not found. Expected: {TRAIN_DIR} and {VAL_DIR}")

# classes & counts
classes = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
num_classes = len(classes)
print(f"Detected {num_classes} classes: {classes}")

# -------------------- augmentations --------------------
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# Try RandAugment if torchvision supports it; otherwise skip
try:
    randaug = transforms.RandAugment(num_ops=2, magnitude=9)
except Exception:
    randaug = None

train_transforms = [
    transforms.Resize(int(IMG_SIZE * 1.12)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(25),
]
if randaug is not None:
    train_transforms.append(randaug)

train_transforms += [
    transforms.ColorJitter(brightness=0.18, contrast=0.12, saturation=0.12, hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
    transforms.RandomErasing(p=0.2, scale=(0.01, 0.12))
]
train_transform = transforms.Compose(train_transforms)

val_transform = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.12)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])

# -------------------- datasets & loaders --------------------
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_transform)
print("Train samples:", len(train_dataset), "Val samples:", len(val_dataset))

# class imbalance handling
y_train = [label for _, label in train_dataset.samples]
if num_classes > 1 and len(y_train) > 0:
    class_weights_np = compute_class_weight(class_weight='balanced', classes=np.arange(num_classes), y=y_train)
    class_weights = torch.tensor(class_weights_np, dtype=torch.float).to(device)
    sample_weights = np.array([class_weights_np[y] for y in y_train], dtype=np.float32)
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
else:
    class_weights = None
    sampler = None

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                           sampler=sampler if sampler is not None else None,
                                           shuffle=(sampler is None),
                                           num_workers=NUM_WORKERS, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                                         num_workers=NUM_WORKERS, pin_memory=True)

# -------------------- model creation (fallback logic) --------------------
def create_model(primary, fallback):
    try:
        m = timm.create_model(primary, pretrained=True, num_classes=num_classes)
        print("Created model:", primary)
        return m, primary
    except Exception as e:
        print(f"Primary model {primary} creation failed ({e}). Falling back to {fallback}.")
        m = timm.create_model(fallback, pretrained=True, num_classes=num_classes)
        return m, fallback

model, used_model = create_model(PREFERRED_MODEL, FALLBACK_MODEL)
model.to(device)

# freeze all, then unfreeze head
for n, p in model.named_parameters():
    p.requires_grad = False

head_unfreeze_count = 0
for name, p in model.named_parameters():
    if ("head" in name) or ("classifier" in name) or name.endswith("fc") or "proj" in name:
        p.requires_grad = True
        head_unfreeze_count += 1
if head_unfreeze_count == 0:  # fallback: try to unfreeze last linear
    for name, p in list(model.named_parameters())[-20:]:
        p.requires_grad = True

print("Trainable head param groups approx:", head_unfreeze_count)

# -------------------- loss + augmentation utilities --------------------
def get_criterion():
    try:
        return nn.CrossEntropyLoss(weight=class_weights, label_smoothing=LABEL_SMOOTHING)
    except TypeError:
        return nn.CrossEntropyLoss(weight=class_weights)

criterion = get_criterion()

def mixup_data(x, y, alpha=MIXUP_ALPHA):
    if alpha <= 0:
        return x, y, None, 1.0
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    if batch_size == 1:
        return x, y, None, 1.0
    idx = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[idx, :]
    return mixed_x, y, y[idx], lam

def cutmix_data(x, y, alpha=CUTMIX_ALPHA):
    if alpha <= 0:
        return x, y, None, 1.0
    lam = np.random.beta(alpha, alpha)
    batch_size, c, h, w = x.size()
    idx = torch.randperm(batch_size).to(x.device)
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(w * cut_rat)
    cut_h = int(h * cut_rat)
    cx = np.random.randint(w)
    cy = np.random.randint(h)
    x1 = np.clip(cx - cut_w // 2, 0, w)
    y1 = np.clip(cy - cut_h // 2, 0, h)
    x2 = np.clip(cx + cut_w // 2, 0, w)
    y2 = np.clip(cy + cut_h // 2, 0, h)
    x[:, :, y1:y2, x1:x2] = x[idx, :, y1:y2, x1:x2]
    lam_adj = 1. - ((x2 - x1) * (y2 - y1) / float(w * h))
    return x, y, y[idx], lam_adj

def mixup_criterion(preds, y_a, y_b, lam):
    return lam * criterion(preds, y_a) + (1 - lam) * criterion(preds, y_b)

# -------------------- training / validation utils --------------------
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

def human_gpu_mem():
    if not torch.cuda.is_available():
        return ""
    return f"{torch.cuda.memory_reserved(device)/1024**3:.2f}GB"

def compute_metrics_all(y_true_list, y_pred_list, y_prob_array, class_names):
    y_true = np.array(y_true_list); y_pred = np.array(y_pred_list)
    acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=class_names, zero_division=0)
    n = len(class_names)
    y_true_bin = label_binarize(y_true, classes=list(range(n)))
    if y_true_bin.ndim == 1:
        y_true_bin = y_true_bin.reshape(-1, 1)
    if y_true_bin.shape[1] == 1:
        y_true_bin = np.hstack([1 - y_true_bin, y_true_bin])
    fpr, tpr, roc_auc = {}, {}, {}
    aucs = []; missing = []
    for i in range(n):
        if np.sum(y_true_bin[:, i]) == 0:
            missing.append(class_names[i]); continue
        try:
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob_array[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i]); aucs.append(roc_auc[i])
        except Exception:
            missing.append(class_names[i])
    macro_auc = float(np.mean(aucs)) if len(aucs) > 0 else 0.0
    return acc, report, roc_auc, fpr, tpr, macro_auc, missing

# single-batch TTA
def tta_predict_batch(model, imgs):
    model.eval()
    probs_sum = None
    with torch.no_grad():
        for tfn in TTA_TRANSFORMS:
            aug = tfn(imgs)
            logits = model(aug)
            probs = F.softmax(logits, dim=1).cpu().numpy()
            probs_sum = probs if probs_sum is None else probs_sum + probs
    return probs_sum / len(TTA_TRANSFORMS)

# train one epoch
def train_one_epoch(model, loader, optimizer, epoch, total_epochs, scheduler=None):
    model.train()
    running_loss = 0.0; correct = 0; total = 0
    pbar = tqdm(enumerate(loader, start=1), total=len(loader),
                desc=f"Train {epoch}/{total_epochs}", ncols=120, leave=False)
    for batch_idx, (imgs, labels) in pbar:
        imgs = imgs.to(device); labels = labels.to(device)
        # randomly apply cutmix or mixup
        if np.random.rand() < CUTMIX_PROB:
            imgs_m, y_a, y_b, lam = cutmix_data(imgs.clone(), labels, alpha=CUTMIX_ALPHA)
        else:
            imgs_m, y_a, y_b, lam = mixup_data(imgs, labels, alpha=MIXUP_ALPHA)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            outputs = model(imgs_m)
            if y_b is None:
                loss = criterion(outputs, labels)
            else:
                loss = mixup_criterion(outputs, y_a, y_b, lam)
            loss = loss / ACCUMULATION_STEPS
        scaler.scale(loss).backward()
        if MAX_GRAD_NORM is not None:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        scaler.step(optimizer); scaler.update(); optimizer.zero_grad()
        if scheduler is not None and isinstance(scheduler, OneCycleLR):
            scheduler.step()
        running_loss += loss.item() * imgs.size(0) * ACCUMULATION_STEPS
        preds = outputs.detach().argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)
        running_epoch_loss = running_loss / total if total>0 else running_loss
        running_epoch_acc = correct / total if total>0 else 0.0
        postfix = {"b_loss": f"{loss.item()*ACCUMULATION_STEPS:.4f}", "e_loss": f"{running_epoch_loss:.4f}", "e_acc": f"{running_epoch_acc:.4f}"}
        if torch.cuda.is_available():
            postfix["GPU"] = human_gpu_mem()
        pbar.set_postfix(postfix, refresh=False)
    pbar.close()
    avg_loss = running_loss / len(loader.dataset) if len(loader.dataset)>0 else 0.0
    return avg_loss, correct / total if total>0 else 0.0

def validate(model, loader, epoch=None):
    model.eval()
    running_loss = 0.0; all_preds=[]; all_labels=[]; all_probs=[]
    pbar = tqdm(enumerate(loader, start=1), total=len(loader),
                desc=f"Val {epoch}" if epoch else "Val", ncols=120, leave=False)
    with torch.no_grad():
        for batch_idx, (imgs, labels) in pbar:
            imgs = imgs.to(device); labels = labels.to(device)
            probs = tta_predict_batch(model, imgs)  # (batch, num_classes) numpy
            preds = np.argmax(probs, axis=1)
            # loss compute requires logits => forward
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(imgs)
                loss = criterion(logits, labels)
            running_loss += loss.item() * imgs.size(0)
            all_probs.append(probs); all_preds.extend(list(preds)); all_labels.extend(list(labels.cpu().numpy()))
            batch_acc = (preds == labels.cpu().numpy()).mean()
            running_epoch_loss = running_loss / (len(all_labels) if len(all_labels)>0 else 1)
            running_epoch_acc = accuracy_score(all_labels, all_preds) if len(all_labels)>0 else 0.0
            postfix = {"b_loss": f"{loss.item():.4f}", "b_acc": f"{batch_acc:.3f}", "e_loss": f"{running_epoch_loss:.4f}", "e_acc": f"{running_epoch_acc:.4f}"}
            if torch.cuda.is_available():
                postfix["GPU"] = human_gpu_mem()
            pbar.set_postfix(postfix, refresh=False)
    pbar.close()
    val_loss = running_loss / len(loader.dataset) if len(loader.dataset)>0 else 0.0
    all_probs = np.vstack(all_probs) if all_probs else np.zeros((0, num_classes))
    return val_loss, all_labels, all_preds, all_probs

# -------------------- training loop --------------------
history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[], "val_auc_macro":[]}
best_score = -1e9
total_epochs = PHASE1_EPOCHS + PHASE2_EPOCHS
stopping_counter = 0

# Phase 1: head-only training
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=HEAD_LR, weight_decay=WEIGHT_DECAY)
steps_per_epoch = max(1, len(train_loader))
scheduler = OneCycleLR(optimizer, max_lr=HEAD_LR, total_steps=PHASE1_EPOCHS * steps_per_epoch)

print("Starting Phase 1 (head-only)")
for epoch in range(1, PHASE1_EPOCHS + 1):
    t0 = time.time()
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, epoch, total_epochs, scheduler=scheduler)
    val_loss, y_true, y_pred, y_prob = validate(model, val_loader, epoch)
    _, _, _, _, _, val_auc_macro, _ = compute_metrics_all(y_true, y_pred, y_prob, classes)
    history["train_loss"].append(tr_loss); history["train_acc"].append(tr_acc)
    history["val_loss"].append(val_loss); history["val_acc"].append(accuracy_score(y_true, y_pred)); history["val_auc_macro"].append(val_auc_macro)
    combined = -val_loss + val_auc_macro
    print(f"Epoch {epoch}/{PHASE1_EPOCHS} summary: tr_acc={tr_acc:.4f} val_acc={history['val_acc'][-1]:.4f} val_loss={val_loss:.4f} val_auc={val_auc_macro:.4f} time={(time.time()-t0):.1f}s")
    if combined > best_score:
        best_score = combined
        torch.save(model.state_dict(), CHECKPOINT_PATH)
        print("Saved best (phase1) to:", CHECKPOINT_PATH); stopping_counter = 0
    else:
        stopping_counter += 1
        print(f"No improvement: {stopping_counter}/{EARLY_STOPPING_PATIENCE}")
    if stopping_counter >= EARLY_STOPPING_PATIENCE:
        print("Early stopping Phase 1"); break

# Phase 2: unfreeze last N_UNFREEZE blocks if ViT-like; otherwise unfreeze most
print("Unfreezing model segments for finetuning")
try:
    if hasattr(model, "blocks"):
        total_blocks = len(model.blocks)
        for i in range(max(0, total_blocks - N_UNFREEZE), total_blocks):
            for p in model.blocks[i].parameters():
                p.requires_grad = True
    elif hasattr(model, "transformer") and hasattr(model.transformer, "blocks"):
        total_blocks = len(model.transformer.blocks)
        for i in range(max(0, total_blocks - N_UNFREEZE), total_blocks):
            for p in model.transformer.blocks[i].parameters():
                p.requires_grad = True
    else:
        # CNN style model: unfreeze last layers and classifier
        ct = 0
        for name, p in reversed(list(model.named_parameters())):
            p.requires_grad = True
            ct += 1
            if ct > 200:  # don't unfreeze everything if very large
                break
except Exception:
    for p in model.parameters():
        p.requires_grad = True

# finetune optimizer & scheduler
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=FINETUNE_LR, weight_decay=WEIGHT_DECAY)
steps_per_epoch = max(1, len(train_loader))
scheduler = OneCycleLR(optimizer, max_lr=FINETUNE_LR, total_steps=PHASE2_EPOCHS * steps_per_epoch)

# SWA preparation
use_swa = True
swa_model = AveragedModel(model) if use_swa else None
swa_start = max(PHASE1_EPOCHS, total_epochs - SWA_FINISH_EPOCHS)

print("Starting Phase 2 (finetune)")
stopping_counter = 0
for epoch in range(PHASE1_EPOCHS + 1, PHASE1_EPOCHS + PHASE2_EPOCHS + 1):
    t0 = time.time()
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, epoch, total_epochs, scheduler=scheduler)
    val_loss, y_true, y_pred, y_prob = validate(model, val_loader, epoch)
    _, _, _, _, _, val_auc_macro, _ = compute_metrics_all(y_true, y_pred, y_prob, classes)
    history["train_loss"].append(tr_loss); history["train_acc"].append(tr_acc)
    history["val_loss"].append(val_loss); history["val_acc"].append(accuracy_score(y_true, y_pred)); history["val_auc_macro"].append(val_auc_macro)

    if use_swa and epoch >= swa_start:
        swa_model.update_parameters(model)

    combined = -val_loss + val_auc_macro
    print(f"Epoch {epoch}/{total_epochs} summary: tr_acc={tr_acc:.4f} val_acc={history['val_acc'][-1]:.4f} val_loss={val_loss:.4f} val_auc={val_auc_macro:.4f} time={(time.time()-t0):.1f}s")
    if combined > best_score:
        best_score = combined; torch.save(model.state_dict(), CHECKPOINT_PATH)
        print("Saved best model to:", CHECKPOINT_PATH); stopping_counter = 0
    else:
        stopping_counter += 1; print(f"No improvement: {stopping_counter}/{EARLY_STOPPING_PATIENCE}")
    if stopping_counter >= EARLY_STOPPING_PATIENCE:
        print("Early stopping Phase 2"); break

# finalize SWA if used
final_model = model
if use_swa and swa_model is not None:
    print("Applying SWA parameters & batchnorm update")
    torch.optim.swa_utils.update_bn(train_loader, swa_model)
    final_model = swa_model.module if hasattr(swa_model, "module") else swa_model

# load best checkpoint to final_model for evaluation
print("Loading best checkpoint for final evaluation:", CHECKPOINT_PATH)
ckpt = torch.load(CHECKPOINT_PATH, map_location=device)
final_model.load_state_dict(ckpt)
final_model.to(device); final_model.eval()

val_loss, y_true, y_pred, y_prob = validate(final_model, val_loader, epoch="final")
acc, report, roc_auc_dict, fpr_dict, tpr_dict, macro_auc, missing = compute_metrics_all(y_true, y_pred, y_prob, classes)
print(f"\nFINAL VAL ACC = {acc*100:.2f}%")
print("\nClassification Report:\n", report)
print("Macro AUC:", macro_auc)
if missing:
    print("Skipped ROC for classes:", missing)

# save final model
torch.save(final_model.state_dict(), FINAL_MODEL_PATH)
print("Saved final model to:", FINAL_MODEL_PATH)

# save simple plots
os.makedirs(GRAPH_DIR, exist_ok=True)
epochs_range = range(1, len(history["train_loss"]) + 1)
plt.figure(figsize=(8,5))
plt.plot(epochs_range, history["train_acc"], marker='o', label='Train Acc')
plt.plot(epochs_range, history["val_acc"], marker='o', label='Val Acc')
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.title("Accuracy over Epochs"); plt.grid(True, linestyle='--', alpha=0.4); plt.legend()
plt.savefig(os.path.join(GRAPH_DIR, "accuracy.png"), dpi=150); plt.close()
plt.figure(figsize=(8,5))
plt.plot(epochs_range, history["train_loss"], marker='o', label='Train Loss')
plt.plot(epochs_range, history["val_loss"], marker='o', label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Loss over Epochs"); plt.grid(True, linestyle='--', alpha=0.4); plt.legend()
plt.savefig(os.path.join(GRAPH_DIR, "loss.png"), dpi=150); plt.close()
cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
plt.figure(figsize=(7,6)); sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.title("Confusion Matrix"); plt.tight_layout()
plt.savefig(os.path.join(GRAPH_DIR, "confusion_matrix.png"), dpi=150); plt.close()

print("Training complete. Plots saved to:", GRAPH_DIR)


Mounted at /content/drive
Extracted dataset to /content/dataset
Device: cuda
Detected 2 classes: ['benign', 'malignant']
Train samples: 2637 Val samples: 660


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Created model: vit_base_patch16_224
Trainable head param groups approx: 28
Starting Phase 1 (head-only)


/tmp/ipython-input-1064087535.py:239: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
Train 1/38:   0%|                                                                               | 0/110 [00:00<?, ?it/s]/tmp/ipython-input-1064087535.py:294: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/doc

...

Epoch 37/38 summary: tr_acc=0.7986 val_acc=0.9561 val_loss=0.2101 val_auc=0.9920 time=58.6s
No improvement: 2/10


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Train 38/38:   0%|                                                                              | 0/110 [00:00<?, ?it/s]/tmp/ipython-input-1064087535.py:294: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Val 38:   0%|                                                                                    | 0/28 [00:00<?, ?it/s]/tmp/ipython-input-1064087535.py:333: FutureWarning: `torch.cuda.amp.autocast

Epoch 38/38 summary: tr_acc=0.7964 val_acc=0.9561 val_loss=0.2101 val_auc=0.9920 time=59.7s
No improvement: 3/10
Applying SWA parameters & batchnorm update
Loading best checkpoint for final evaluation: /content/drive/MyDrive/best_model_4k.pt


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Val final:   0%|                                                                                 | 0/28 [00:00<?, ?it/s]/tmp/ipython-input-1064087535.py:333: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
Val final:   4%|▏   | 1/28 [00:01<00:38,  1.42s/it, b_loss=0.2956, b_acc=0.875, e_loss=0.2956, e_acc=0.8750, GPU=2.85GB]/tmp/ipython-input-1064087535.py:333: FutureWarning: `torch.cuda.amp.autocast


FINAL VAL ACC = 96.21%

Classification Report:
               precision    recall  f1-score   support

      benign       0.97      0.96      0.97       360
   malignant       0.95      0.96      0.96       300

    accuracy                           0.96       660
   macro avg       0.96      0.96      0.96       660
weighted avg       0.96      0.96      0.96       660

Macro AUC: 0.9931759259259259
Saved final model to: /content/drive/MyDrive/final_model_4k.pt
Training complete. Plots saved to: /content/drive/MyDrive/results_4k
